# Query a CSV File using OpenAI LLM with LangChain

This project demonstrates how to use OpenAI's LLM with LangChain to query data from a CSV file. It involves creating a retrieval-based system that processes the CSV content, converts it into a vectorized format, and allows natural language queries to extract meaningful insights.

---

## Features

- **CSV Parsing**: Automatically reads and processes the CSV file.
- **OpenAI LLM Integration**: Uses OpenAI's powerful language model to interpret and respond to natural language queries.
- **Vectorization**: Transforms CSV data into vector embeddings for efficient querying.
- **LangChain Framework**: Leverages LangChain's tools to simplify the retrieval and response generation process.

---

## Prerequisites

1. **Python**: Ensure Python 3.8 or above is installed.
2. **OpenAI API Key**: Obtain an API key from [OpenAI](https://platform.openai.com/).
3. **Required Libraries**: Install the necessary dependencies:
   ```bash
   pip install langchain openai pandas tiktoken faiss-cpu


In [20]:
import os
import numpy as np
import pandas as pd

import openai

from dotenv import load_dotenv
load_dotenv()

from langchain.document_loaders import csv_loader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings


In [14]:
# Retrieve the API key from .env file
openai_api_key = os.getenv('OPENAI_API_KEY')

# setting the API key
openai.api_key = openai_api_key

In [53]:
# Load the data
loader = csv_loader.CSVLoader(file_path='data/titanic.csv')
loader_1 = csv_loader.CSVLoader(file_path='data/pokemon.csv')
loader

In [71]:
# Initialize the embedding model
embedding_model = OpenAIEmbeddings()

# Initialize the index creator with the embedding model
index_creator = VectorstoreIndexCreator(embedding=embedding_model)

docsearch = index_creator.from_loaders(loaders=[loader,loader_1])

c:\Users\sathw\Desktop\ML\LLM\LLM_Playground\venv\lib\site-packages\langchain\indexes\vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [72]:
# Initialize the OpenAI LLM
# llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

# Create the RetrievalQA chain
chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),  # Pass the initialized LLM instance, not the class
    chain_type='stuff',
    retriever=docsearch.vectorstore.as_retriever(),  # Ensure docsearch is properly set up
    input_key='question'  # Specify the input key for your chain
)

In [77]:
pokemon = pd.read_csv('data/pokemon.csv')
pokemon.sort_values(by='Attack', ascending=False)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
163,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True
232,214,HeracrossMega Heracross,Bug,Fighting,600,80,185,115,40,105,75,2,False
429,386,DeoxysAttack Forme,Psychic,NaN,600,50,180,20,180,20,150,3,True
426,384,RayquazaMega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True
424,383,GroudonPrimal Groudon,Ground,Fire,770,100,180,160,150,90,90,3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,213,Shuckle,Bug,Rock,505,20,10,230,10,230,5,2,False
261,242,Blissey,Normal,NaN,540,255,10,10,75,135,55,2,False
139,129,Magikarp,Water,NaN,200,20,10,55,15,20,80,1,False
121,113,Chansey,Normal,NaN,450,250,5,5,35,105,50,1,False


In [74]:
query = "What is Hoothoot stats"

# Run the chain
response = chain({'question': query})

print(response['result'])

 Hoothoot has a total stat of 262, with 60 HP, 30 Attack, 30 Defense, 36 Special Attack, 56 Special Defense, and 50 Speed. It is a Normal and Flying type Pokemon from Generation 2, and is not a Legendary Pokemon.


In [76]:
query = "Which pokemon has the maximum attack?"

# Run the chain
response = chain({'question': query})

print(response['result'])


Mega Rayquaza has the maximum attack with a base stat of 180.


In [56]:
query = "What are the columns in the Titanic dataset?"

# Run the chain
response = chain({'question': query})

print(response['result'])

 The columns in the Titanic dataset are PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, and Embarked.


In [44]:
query = "What is the maximum Age in the Titanic dataset?"

# Run the chain
response = chain({'question': query})

print(response['result'])

 The maximum Age in the Titanic dataset is 71.


In [ ]:
query = "how many pclass are there in the titanic dataset?"

# Run the chain
response = chain({'question': query})

print(response['result'])

In [49]:
query = "what is the maximum Fare in the titanic dataset?"

# Run the chain
response = chain({'question': query})

print(response['result'])

 I don't know.


In [45]:
query = "What is the Average Age in the Titanic dataset?"

# Run the chain
response = chain({'question': query})

print(response['result'])

 The average age in the Titanic dataset cannot be determined solely based on the given information, as there are missing values for age in some of the passenger records.


In [46]:
data = pd.read_csv('data/titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [51]:
query = "distinct Cabin names in the titanic dataset?"

# Run the chain
response = chain({'question': query})

print(response['result'])


There are 28 distinct Cabin names in the Titanic dataset. 


In [52]:
data['Cabin'].nunique()

147